## Tensor基础

### 2.1 Tensor

In [11]:
import torch

#### 基本构造方法

In [2]:
x = torch.Tensor(2,3)
x

tensor([[-8.1173e+31,  2.0361e-42,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00]])

In [3]:
list = [[0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1]]
fromList = torch.tensor(list)
fromList

tensor([[0, 0, 0],
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]])

#### 快速创建方法
- 全0 or 全1
- 对角1
- rand: 随机[0, 1)
- arange: 区间 + optional 步长

In [4]:
print(torch.zeros(2,3,2))
print(torch.ones(2,3))
print(torch.eye(2,3))
print(torch.arange(1, 4, 0.5))

tensor([[[0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.]]])
tensor([[1., 1., 1.],
        [1., 1., 1.]])
tensor([[1., 0., 0.],
        [0., 1., 0.]])
tensor([1.0000, 1.5000, 2.0000, 2.5000, 3.0000, 3.5000])


#### 常见属性与操作

In [5]:
print(x.type())
print(x.dtype)
print('fromList shape is ' + str(fromList.shape))
print(fromList[0][0]) # a tensor


torch.FloatTensor
torch.float32
fromList shape is torch.Size([4, 3])
tensor(0)


#### 常见数学操作
**static and non-static method can both apply in some case** :grinning:

如果操作带*下划线*, 返回值将会覆盖对象 :thinking:

- `add()`, `mul()`, `div()`: 同操作一个标量, 或者Tensor之间逐个元素操作
- `fmod()`, `remainder()`: 取余数
- `ceil()`, `floor()`, `log()`, `exp()`, `sigmoid()`, `sqrt()`, `abs()`, `min()`, `max()`: 懂得都懂
- `clamp()`: 至少传入一个标量, 取上下限
- `round()`: 取最近的整数
- `frac()`: 取小数部分
- `neg()`, `reciprocal()`: 取负, 取倒数
- `pow()`: 取幂, 必要时LLM
- `sign()`: +-1显示正负
- `dist()`：返回两个Tensor之间的距离(范数)
- `norm()`: 计算Tensor范数
- `sum()`, `prod()`: 返回所有元素之和/积
- `torch.mv()`, `torch.mm()`: matrix/vector乘法

In [10]:
l = [-1, +1, -0.255, +6.12]
l1 = torch.tensor(l)
print(l1.sign())
print(l1.dist(l1))

tensor([-1.,  1., -1.,  1.])
tensor(0.)


- 爱因斯坦求和约定 `einsum()`

In [ ]:
import torch

z_norm = torch.randn(2, 3, 4)

# 使用 torch.einsum 计算相似性矩阵
z_cos_sim = torch.einsum('bci,bcj->bij', z_norm, z_norm)

print(z_cos_sim.shape)  # 输出：torch.Size([2, 4, 4])

`torch.einsum` 是 PyTorch 中基于 **爱因斯坦求和约定**（Einstein Summation Convention）实现的函数，用于高效、简洁地表达多维张量的复杂运算。它能够替代矩阵乘法、转置、求和、点积、外积等多种操作，尤其适合处理高维张量的操作。

---


爱因斯坦求和约定通过省略求和符号（`Σ`），用下标标记张量的维度，直接表达乘积和求和的规则。例如：
- 矩阵乘法 `C = A @ B` 可表示为 `ik,kj->ij`，含义是对中间维度 `k` 求和。
- 向量内积 `a·b` 可表示为 `i,i->`，即对相同下标 `i` 的乘积求和。

---

### **`torch.einsum` 语法**
```python
torch.einsum(equation, *operands)
```
- **`equation`**: 字符串，描述操作的维度规则（如 `"ik,kj->ij"`）。
- **`operands`**: 输入张量，数量与方程中的输入数量一致。

---

### **关键规则**
1. **下标字符**：用字母（如 `i, j, k`）标记维度。
2. **逗号分隔输入**：输入张量的维度用逗号分隔（如 `"ik,kj"` 表示两个输入）。
3. **箭头后为输出**：`->` 后的字符表示输出维度。
4. **隐式求和**：在方程中出现但未在输出中标记的下标会被求和。

---

### **常见用例与示例**

#### 1. **矩阵乘法**
```python
A = torch.randn(2, 3)
B = torch.randn(3, 4)
C = torch.einsum("ik,kj->ij", A, B)  # 等价于 A @ B
```

#### 2. **逐元素乘法**
```python
A = torch.randn(2, 3)
B = torch.randn(2, 3)
C = torch.einsum("ij,ij->ij", A, B)  # 等价于 A * B
```

#### 3. **求和**
```python
A = torch.randn(2, 3)
sum_all = torch.einsum("ij->", A)      # 所有元素求和，等价于 A.sum()
sum_row = torch.einsum("ij->i", A)    # 按行求和，等价于 A.sum(dim=1)
```

#### 4. **转置**
```python
A = torch.randn(2, 3)
A_T = torch.einsum("ij->ji", A)  # 等价于 A.T 或 A.permute(1,0)
```

#### 5. **对角线元素**
```python
A = torch.randn(3, 3)
diag = torch.einsum("ii->i", A)  # 提取对角线元素，等价于 A.diag()
```

#### 6. **批量矩阵乘法**
```python
A = torch.randn(5, 2, 3)  # batch=5
B = torch.randn(5, 3, 4)
C = torch.einsum("bij,bjk->bik", A, B)  # 等价于 A @ B
```

#### 7. **外积**
```python
a = torch.randn(3)
b = torch.randn(4)
outer = torch.einsum("i,j->ij", a, b)  # 外积，等价于 torch.outer(a, b)
```

#### 8. **张量缩并（Tensor Contraction）**
```python
A = torch.randn(3, 4, 5)
B = torch.randn(5, 3, 6)
C = torch.einsum("ijk,kil->jl", A, B)  # 缩并维度 k 和 i
```

---

### **广播机制**
`einsum` 支持自动广播。例如，向量与矩阵的乘法：
```python
a = torch.randn(3)
B = torch.randn(5, 3, 4)
C = torch.einsum("i, b i j -> b j", a, B)  # 结果形状 (5, 4)
```

---

### **注意事项**
1. **效率**：对于简单操作（如矩阵乘法），直接使用内置函数（如 `torch.matmul`）可能更高效。
2. **维度匹配**：输入张量的对应维度大小必须一致，否则报错。
3. **可读性**：复杂的方程可能降低代码可读性，建议添加注释。

---


#### 常见线性代数数学操作
kind of like non-static in torch
- `mv()`, `mm()`: matrix/vector乘法
- `dot()`: 点乘
- `addmm()`, `addmv()`, `addr()`: 两个Tensor操作后与其中一个相加
- `bmm()`, `addbmm()`, `baddbmm()`: 不常见, 必要时LLM
- `eig()`: 求特征值和特征向量
- `ger()`: 求两个向量的张量积
- `inverse()`: 求逆
- `t()`: 转置, 仅限二维矩阵

#### 连接和切片
torch non-static
- `cat()`: 第一个参数将所有需要连接的Tensor组成一个元组传入, 第二个参数是连接维度
- `chunk()`: 第一被切的对象, 第二切分块数, 第三切分维度
- `index_select()`: 类比numpy dataframe的切片
- `unbind()`: 沿着给定维度进行逐个单位拆分, 返回序列
- `split()`: 切成相等形状的块
- `nonzero()`: 返回非零索引的Tensor
- `squeeze()`: 除掉维数1
- `unsqueeze()`: 指定维度添加1
- `stack()`: 和`cat()`不同在于会添加维度
- `transpose()`: 对Tensor指定的两个维度进行转置

In [14]:
x = torch.randn(3, 4)
print(x)
indices = torch.tensor([0, 2])
selected_rows = torch.index_select(x, 0, indices)
selected_rows

tensor([[-1.3588, -1.2730,  1.0648,  0.7868],
        [-1.4120, -0.7000, -0.5600,  0.8299],
        [ 0.0598,  0.4924, -0.7204, -1.2977]])


tensor([[-1.3588, -1.2730,  1.0648,  0.7868],
        [ 0.0598,  0.4924, -0.7204, -1.2977]])

In [17]:
c = torch.tensor([[1, 2], [3, 0], [4, 5]])
nonzero_and_greater_than_2 = torch.nonzero(c > 2)
print(nonzero_and_greater_than_2)  # 输出: tensor([[1, 0],
                                  #              [2, 0],
                                  #              [2, 1]])

tensor([[1, 0],
        [2, 0],
        [2, 1]])


In [18]:
x = torch.tensor([[[1], [2], [3]]])  # 形状为 (1, 3, 1)
y = torch.squeeze(x)  # 移除所有大小为 1 的维度，形状变为 (3,)
print(y)
z = torch.squeeze(x, dim=0) # 指定移除第一个维度，形状变为 (3, 1)
print(z)

tensor([1, 2, 3])
tensor([[1],
        [2],
        [3]])


In [19]:

# 假设我们有两个形状为 (2, 3) 的张量
x = torch.tensor([[1, 2, 3], [4, 5, 6]])
y = torch.tensor([[7, 8, 9], [10, 11, 12]])

# 沿着一个新的维度堆叠它们，结果的形状将是 (2, 2, 3)
z = torch.stack((x, y), dim=0)
print(z)

tensor([[[ 1,  2,  3],
         [ 4,  5,  6]],

        [[ 7,  8,  9],
         [10, 11, 12]]])


#### 变形
- `view()`: 变形, 当传入-1时会自动计算维度
- `flatten()`: 展开, 侧重于沿着指定维度展开
- `einops.rearrange()`：变形，支持优雅的变形操作

In [21]:
x = torch.rand(2, 3, 4)
y = x.view(2,12)
print(y.shape)

torch.Size([2, 12])


In [22]:
# 创建一个形状为 (2, 3, 4) 的张量
x = torch.arange(24).reshape(2, 3, 4)

# 将整个张量展平成一维张量
y = torch.flatten(x)
print(y)  # 输出: tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23])

# 展平从第二个维度开始的所有元素
z = torch.flatten(x, start_dim=1)
print(z)

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23])
tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11],
        [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]])


In [ ]:
from einops import rearrange

z_flat = rearrange(z, 'b c h w -> b c (h w)')

#### CUDA加速

In [26]:
x = x.cuda()
print(x.device)
x = x.to('cpu')
print(x.device)

cuda:0
cpu


##### 内存布局
- 配合 `view()` 或 `reshape()`：`view()` 和 `reshape()` 需要张量在内存中是连续的，否则会报错。如果张量经过 `transpose()` 或 `permute()` 等操作后变得不连续，可以调用 `contiguous()` 使其连续。
- 某些操作（如矩阵乘法、卷积等）需要张量在内存中连续存储以提高计算效率



In [ ]:
import torch

# 创建一个张量
x = torch.tensor([[1, 2, 3], [4, 5, 6]])

# 对张量进行转置操作，导致内存不连续
y = x.t()

# 检查是否连续
print(y.is_contiguous())  # 输出：False

# 使用 contiguous() 使其连续
z = y.contiguous(memory_format=torch.contiguous_format) # 行优先

# 检查是否连续
print(z.is_contiguous())  # 输出：True

False
True
